In [3]:
import pymysql

conn = pymysql.connect(host='13.125.102.154', user='hello', password='Csedbadmin!1', db='pytest')
curs = conn.cursor()

raspberryPiID = input("라즈베리파이의 ID를 입력: ")
raspberryPiPW = input('비밀번호를 입력: ')
name = input('냉장고 이름을 입력: ')
location = input('냉장고의 위치를 입력: ')
cameraID = input('카메라의 ID를 입력: ')

sql = "insert into refrigerator values(%s, %s, %s, %s)"
curs.execute(sql, (raspberryPiID, raspberryPiPW, name, location))
conn.commit()

sql = "insert into camera values(%s, %s)"
curs.execute(sql, (cameraID, raspberryPiID))
conn.commit()

with open(filename, mode = 'w') as f:
    f.write(raspberryPiID + ' ' + cameraID)
    
with open(filename, mode = 'r') as f:
    print(f.read())
conn.close()

라즈베리파이의 ID를 입력: 1
비밀번호를 입력: 1
냉장고 이름을 입력: 1
냉장고의 위치를 입력: 1
카메라의 ID를 입력: 1
(('1', '1', '1', '1'), ('applemango', 'applemango!1', '졸프용 냉장고', '한성대'), ('test1', 'test1!', '테스트용', '집'), ('test2', 'test2', '테스트', '집'), ('test3', 'test3', 'home', 'home'))
(('1', '1'), ('camera1', 'applemango'), ('camera2', 'test2'), ('camera3', 'test3'))
1 1


In [7]:
import pymysql

conn = pymysql.connect(host='13.125.102.154', user='hello', password='Csedbadmin!1', db='pytest')
curs = conn.cursor()

sql = "select photoID from photo where cameraID=%s"
curs.execute(sql, 'camera1')
rows = curs.fetchall()
pre = ''
if len(rows) != 0:
    pre = rows[0][0];       
    pre_split = pre.split('_')            
    for row in rows :    
        row_split = row[0].split('_')
        for i in range(0, len(pre_split)) :
            if(int(pre_split[i]) < int(row_split[i])) :
                pre = row[0]
                pre_split = pre.split('_')
                break;
print(pre)

2020_4_21_13_33_45


In [33]:
import pymysql
from datetime import datetime, timedelta

conn = pymysql.connect(host='13.125.102.154', user='hello', password='Csedbadmin!1', db='pytest')
curs = conn.cursor()

sql = "select photoID from photo where cameraID=%s"
curs.execute(sql, 'camera1')
rows = curs.fetchall()
print(rows)

now = '2020_4_21_14_6_32'
now_split = now.split('_')
now_time = datetime(int(now_split[0]), int(now_split[1]), int(now_split[2]), int(now_split[3]), int(now_split[4]), int(now_split[5]))
    
pre = ''
if len(rows) != 0:
    pre = rows[0][0];       
    pre_split = pre.split('_') 
    pre_time = datetime(int(pre_split[0]), int(pre_split[1]), int(pre_split[2]), int(pre_split[3]), int(pre_split[4]), int(pre_split[5]))
    for row in rows :    
        row_split = row[0].split('_')
        row_time = datetime(int(row_split[0]), int(row_split[1]), int(row_split[2]), int(row_split[3]), int(row_split[4]), int(row_split[5]))
        print(pre_time, row_time)
        if now_time == row_time:
            pass
        elif pre_time < row_time :
            pre = row[0]
            pre_split = pre.split('_')
            pre_time = datetime(int(pre_split[0]), int(pre_split[1]), int(pre_split[2]), int(pre_split[3]), int(pre_split[4]), int(pre_split[5]))
print(pre)
conn.close()

()



In [28]:
import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta
import time
import pymysql

googleID = 'applemangoHansung@gmail.com'
googlePW = 'applemango!@#123'

smtp = smtplib.SMTP('smtp.gmail.com', 587) 
smtp.ehlo()      
smtp.starttls()  
smtp.login(googleID, googlePW)

def sendEmail(title, text, sendID = 'cjfwoqkr0105@gmail.com'):
    msg = MIMEText(text)
    msg['Subject'] = title
    msg['To'] = sendID
    smtp.sendmail(googleID, sendID, msg.as_string())

conn = pymysql.connect(host='13.125.102.154', user='hello', password='Csedbadmin!1', db='pytest')
curs = conn.cursor()

preTime = datetime.now()
try:
    while True:
        nowTime = datetime.now()
        print(preTime, nowTime)
            
        # 냉장고 내의 음식의 유통기한이 임박, 초과하면 주인들에게 메일
        # 카메라 마다 최신 사진을 찾은 뒤, 식품의 유통기한과 넣은 시간을 현재 시간과 비교 
        sql = "select * from camera"
        curs.execute(sql)
        rows = curs.fetchall()
        print(rows)
        
        for row in rows:      
            sql = "select photoID from photo where cameraID = '" + row[0] + "'"
            curs.execute(sql)
            photos = curs.fetchall()
            print(photos)
            
            late = ' '
            if len(photos) != 0:
                late = photos[0][0];       
                late_split = late.split('_') 
                late_time = datetime(int(late_split[0]), int(late_split[1]), int(late_split[2]), int(late_split[3]), int(late_split[4]), int(late_split[5]))
                for photo in photos :
                    photo_split = photo[0].split('_')
                    photo_time = datetime(int(photo_split[0]), int(photo_split[1]), int(photo_split[2]), int(photo_split[3]), int(photo_split[4]), int(photo_split[5]))
                    if late_time < photo_time :
                        late = photo[0]
                        late_split = late.split('_')
                        late_time = datetime(int(late_split[0]), int(late_split[1]), int(late_split[2]), int(late_split[3]), int(late_split[4]), int(late_split[5]))
            print(late)
            
            sql = "select * from refrigerator where raspberryPiID = '" + row[1] + "'"
            curs.execute(sql)
            refrigerator = curs.fetchall()
            print(refrigerator)
            
            sql = "select * from possession where raspberryPiID = '" + refrigerator[0][0] + "'"
            curs.execute(sql)
            possessions = curs.fetchall()
            print(possessions)
            
            for possession in possessions:
                email = possession[0]
                
                sql = "select * from memberInfo where email = '" + email + "'"
                curs.execute(sql)
                member = curs.fetchall()
                print(member)
                
                if late != ' ':
                    sql = "select * from photographedFood where photoID = '" + late + "'"
                    curs.execute(sql)
                    foods = curs.fetchall()
                    print(foods)
                    
                    for food in foods:
                        foodName = food[2]
                        shelfLife = food[3]
                        inday_split = food[4].split('_')
                        inTime = datetime(int(inday_split[0]), int(inday_split[1]), int(inday_split[2]), int(inday_split[3]), int(inday_split[4]), int(inday_split[5]))
                        diff = (nowTime-inTime).days
                        # 회원의 초과 메일 설정이 켜져 있고, 해당 식품으로 메일을 보낸 적 없고, 유통기한 초과 시 메일
                        if (member[0][5] == 1) and (diff > shelfLife) : # 보낸 기록 확인 추가
                            # 냉장고 이름 식품명 
                            title = refrigerator[0][2] + ' ' + foodName + ' ' + '유통기한 초과!'
                            text = refrigerator[0][2] + ' ' + foodName + ' ' + '유통기한 초과!'
                            # text에 레시피 추천 추가
                            sendEmail(title, text, email)
                        # 회원의 임박 메일 설정이 켜져 있고, 해당 식품으로 메일을 보낸 적 없고, 유통기한 임박 시 메일
                        elif (member[0][4] == 1) and ((diff >= shelfLife - 3) or (shelfLife < 3)) : # 보낸 기록 확인 추가
                            title = refrigerator[0][2] + ' ' + foodName + ' ' + '유통기한 임박!'
                            text = refrigerator[0][2] + ' ' + foodName + ' ' + '유통기한 임박!'
                            # text에 레시피 추천 추가
                            sendEmail(title, text, email)
                
        # images 하루 한 번 전체 삭제
        if nowTime.hour == 0 and preTime.hour == 23:
            sql = "delete from images"
            curs.execute(sql)
            conn.commit()
        
        # eatenFood 24시간이 지난 음식 삭제
        sql = "select * from eatenFood"
        curs.execute(sql)
        rows = curs.fetchall()
        print(rows)
        
        for row in rows:
            photoID = row[0]
            row_split = photoID.split('_')
            row_time = datetime(int(row_split[0]), int(row_split[1]), int(row_split[2]), int(row_split[3]), int(row_split[4]), int(row_split[5]))
            if (nowTime - row_time).days > 0:
                sql = "delete from eatenFood where photoID = '" + photoID + "'"
                curs.execute(sql)
                conn.commit()
        
        # photo 테이블에 저장한지 30일 초과한 data와 analyzedData 삭제
        sql = "select * from photo"
        curs.execute(sql)
        rows = curs.fetchall()
        
        for row in rows:
            photoID = row[0]
            row_split = photoID.split('_')
            row_time = datetime(int(row_split[0]), int(row_split[1]), int(row_split[2]), int(row_split[3]), int(row_split[4]), int(row_split[5]))
            if (nowTime - row_time).days > 30:
                sql = "update photo set data = 'NULL', analyzedData = 'NULL' where = '" + photoID + "'"
                curs.execute(sql)
                conn.commit()
                
        preTime = nowTime
        
        # 1분에 한번 검사
        time.sleep(60)
except KeyboardInterrupt:
       pass
smtp.quit()
conn.close()

2020-05-18 22:54:09.049058 2020-05-18 22:54:09.050058
(('camera1', 'applemango'),)
(('2020_5_18_18_10_53',), ('2020_5_18_18_11_13',))
2020_5_18_18_11_13
(('applemango', 'applemango!1', '졸프용 냉장고', '한성대'),)
(('cjfwoqkr0105@naver.com', 'applemango'),)
(('cjfwoqkr0105@naver.com', 'qwer1234', '19960105', 'M', 1, 1, None, None, None, None, None, 1, 1, 1, None),)
(('2020_5_18_18_11_13', 1, '토마토', 7, '2020_5_18_18_10_53', 0, 0, 0, 0),)
(('2020_5_18_18_11_13', '토마토'),)
2020-05-18 22:54:09.050058 2020-05-18 22:55:09.183072
(('camera1', 'applemango'),)
(('2020_5_18_18_10_53',), ('2020_5_18_18_11_13',))
2020_5_18_18_11_13
(('applemango', 'applemango!1', '졸프용 냉장고', '한성대'),)
(('cjfwoqkr0105@naver.com', 'applemango'),)
(('cjfwoqkr0105@naver.com', 'qwer1234', '19960105', 'M', 1, 1, None, None, None, None, None, 1, 1, 1, None),)
(('2020_5_18_18_11_13', 1, '토마토', 7, '2020_5_18_18_10_53', 0, 0, 0, 0),)
(('2020_5_18_18_11_13', '토마토'),)
2020-05-18 22:55:09.183072 2020-05-18 22:56:09.326447
(('camera1', 'ap

In [13]:
#설정한 시간에 레시피 추천 메일

import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta
import time
import pymysql

googleID = 'applemangoHansung@gmail.com'
googlePW = 'applemango!@#123'

smtp = smtplib.SMTP('smtp.gmail.com', 587) 
smtp.ehlo()      
smtp.starttls()  
smtp.login(googleID, googlePW)

def sendEmail(title, text, sendID = 'cjfwoqkr0105@naver.com'):
    msg = MIMEText(text)
    msg['Subject'] = title
    msg['To'] = sendID
    smtp.sendmail(googleID, sendID, msg.as_string())

conn = pymysql.connect(host='13.125.102.154', user='hello', password='Csedbadmin!1', db='pytest')
curs = conn.cursor()

nowTime = datetime.now()
print(nowTime)

sql = "select * from members"
curs.execute(sql)
members = curs.fetchall()

for member in members:
    email = member[3]
    sql = "select recipeName from recommendRecipe where email = '" + email + "'"
    curs.execute(sql)
    recipes = curs.fetchall()

    title = '오늘의 레시피 추천'
    text = '오늘의 추천 레시피입니다.\n'
    for recipe in recipes:
        text = text + recipe[0] + ': http://project/recipe/' + recipe[0] + '\n'
    sendEmail(title, text, email)

smtp.quit()
conn.close()

2020-06-01 02:30:49.133063


In [27]:
from datetime import datetime, timedelta
import time


time1_text = '2020_4_13_4_50'
time_split = time1_text.split('_')
time1 = datetime(int(time_split[0]), int(time_split[1]), int(time_split[2]), int(time_split[3]), int(time_split[4]))

time2_text = '2020_4_13_4_50_20'
time_split = time2_text.split('_')
time2 = datetime(int(time_split[0]), int(time_split[1]), int(time_split[2]), int(time_split[3]), int(time_split[4]), int(time_split[5]))
print(time1, time2, time1 == time2)

nowTime = datetime.now()
n = datetime(nowTime.year, nowTime.month, nowTime.day, nowTime.hour, nowTime.minute)
print(time1, n, time1 == n)

2020-04-13 04:50:00 2020-04-13 04:50:20 False
2020-04-13 04:50:00 2020-05-25 02:52:00 False
